In [1]:
import json
import pandas as pd
from openai import OpenAI
import faiss
import os
from tqdm import tqdm
from getpass import getpass
import h5py
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
OPENAI_KEY = getpass("KEY OpenAI")
NOME_MODELO_EMB_OPENAI = "text-embedding-3-large"
DIM_MODELO_EMB_OPENAI = 3072

# Modelos disponíveis
MODELOS_EMB_NOME_E_DIM_EMB = [(NOME_MODELO_EMB_OPENAI, DIM_MODELO_EMB_OPENAI)]

# Modelos para gerar. A ideia é que, uma vez que já foi gerado, pode tirar daqui. Daí ele não precisa carregar o arquivo e ver se está lá
MODELOS_EMB_PARA_GERAR = []

KEY OpenAI ········


In [3]:
ARQUIVO_EMBEDDINGS_CHUNKS = 'outputs/embeddings_chunks.h5'
ARQUIVO_EMBEDDINGS_QUESTOES = 'outputs/embeddings_questoes.h5'

# 1. Carregar as bases de dados de chunks e de questões

In [4]:
def load_jsonl(path):
    with open(path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]

chunks_pesquisa = load_jsonl('inputs/chunks_pesquisa.jsonl')
questoes = load_jsonl('inputs/questoes.jsonl')

Separa as URN/TEXTO dos chunks e as ID/ENUNCIADO das questões

In [5]:
urn_chunks = [c['URN'] for c in chunks_pesquisa]
texto_chunks = [c['TEXTO'] for c in chunks_pesquisa]

id_questoes = [q['ID_QUESTAO'] for q in questoes]
enunciado_questoes = [q['ENUNCIADO_COM_ALTERNATIVAS'] for q in questoes]

# 2. Criar as estruturas em arquivos H5

Garante que o arquivo existe e que tem os datasets nele:

In [6]:
def criar_estrutura_embeddings(arquivo, nome_id, lista_id):
    with h5py.File(arquivo, "a") as f:
        chunk_size=128
        n = len(lista_id)
        
        if nome_id not in f:
            # Quando criar o dataset para as URNS/ID, já cria ele preenchido com todas elas
            f.create_dataset(
                nome_id,
                data=np.array(lista_id, dtype="S"),  # grava tudo de uma vez
                maxshape=(None,),
                dtype=h5py.string_dtype(encoding="utf-8"),
                chunks=True
            )
    
        for nome_modelo, dim_modelo in MODELOS_EMB_NOME_E_DIM_EMB:
            if nome_modelo not in f:
                # Quando criar o dataset com os embeddings do modelo, cria preenchido com nan
                ds = f.create_dataset(
                    nome_modelo,
                    shape=(n, dim_modelo),
                    maxshape=(n, dim_modelo),
                    dtype=np.float16,
                    compression="gzip",
                    chunks=(chunk_size, dim_modelo)
                )
                ds[:] = np.nan

criar_estrutura_embeddings(ARQUIVO_EMBEDDINGS_CHUNKS, 'urn', urn_chunks)
criar_estrutura_embeddings(ARQUIVO_EMBEDDINGS_QUESTOES, 'id', id_questoes)

Funções auxiliares para saber se já existe embedding associado e para atualizar embeddings:

In [7]:
def existe_embedding(arquivo, nome_modelo, idx):
    with h5py.File(arquivo, "a") as f:
        ds = f[nome_modelo]

        return not np.isnan(ds[idx, 0])
    
def atualizar_embedding(arquivo, nome_modelo, idx, embedding):
    with h5py.File(arquivo, "a") as f:
        ds = f[nome_modelo]

        ds[idx] = np.asarray(embedding, dtype=np.float16)

# 2. Cria embeddings para o campo TEXTO (chunks) e para o campo ENUNCIADO_COM_ALTERNATIVAS (questões)

Funções auxiliares para geração de embeddings

In [8]:
def extrai_emb_com_retry(id, texto, nome_modelo, func_get_emb):
    try:
        return func_get_emb(nome_modelo, texto)
    except Exception as e:
        print(f'id: {id}. Chunk muito grande. Reduzindo em 20%')
        print(e.message)
        # Extrai, da mensagem de erro, o total de tokens requisitados e diminui o texto proporcionalmente.
        # No caso da openai, eles aceitam 8192 de entrada.
        # Na hora de diminuir, garante que vai diminuir pelo menos 20% do texto de entrada
        reduzir_para = int(len(texto)*.8)
        if nome_modelo == EMB_MODEL_OPENAI:
            match = re.search(r'requested\s+(\d+)\s+tokens', ex.message)
             # Se não achou a mensagem, considera 8192 para reduzir em 20%
            total_token_requisitados = int(match.group(1)) if match else 8192
            reduzir_para = int(min(8192/total_token_requisitados, 0.8) * len(texto))
            
        return extrai_emb_com_retry(id, texto[:reduzir_para], nome_modelo, func_get_emb)

def extrai_emb_chunks_e_salva(nome_modelo, func_get_emb):
    for chunk in tqdm(chunks_pesquisa):
        urn = chunk['URN']
        texto = chunk['TEXTO']
    
        if nome_modelo not in emb_chunks[urn]:
            emb_chunks[urn][nome_modelo] = extrai_emb_com_retry(urn, texto, nome_modelo, func_get_emb)
            #emb_chunks[urn][nome_modelo] = func_get_emb(nome_modelo, texto)
            salvar_embeddings(NOME_ARQUIVO_EMBEDDINGS_CHUNKS, emb_chunks)

def extrai_emb_questoes_e_salva(nome_modelo, func_get_emb):
    for q in tqdm(questoes):
        id = q['ID_QUESTAO']
        texto = q['ENUNCIADO_COM_ALTERNATIVAS']
    
        if nome_modelo not in emb_questoes[id]:
            emb_questoes[id][nome_modelo] = extrai_emb_com_retry(id, texto, nome_modelo, func_get_emb)
            salvar_embeddings(NOME_ARQUIVO_EMBEDDINGS_QUESTOES, emb_questoes)

Função para extrair embeddings usando o modelo da openAI

In [9]:
client_openai = OpenAI(api_key=OPENAI_KEY, base_url=None)
def extrair_embeddings_openai(nome_modelo, texto):
   return client_openai.embeddings.create(input = [texto], model=nome_modelo).data[0].embedding

Agora gera os embeddings dos chunks:

In [10]:
# Varre todos os urns
for idx, urn in enumerate(tqdm(urn_chunks)):
    texto = texto_chunks[idx]

    for nome_modelo, _ in MODELOS_EMB_PARA_GERAR:
        if not existe_embedding(ARQUIVO_EMBEDDINGS_CHUNKS, nome_modelo, idx):
            # TODO: Depois generalizar isso daqui para não ficar tendo que fazer if com o nome dos modelos
            if nome_modelo == NOME_MODELO_EMB_OPENAI:
                emb_gerado = extrair_embeddings_openai(nome_modelo, texto)
                atualizar_embedding(ARQUIVO_EMBEDDINGS_CHUNKS, nome_modelo, idx, emb_gerado)

100%|██████████████████████████████████████████████████████████████████████████| 7036/7036 [00:00<00:00, 678775.51it/s]


Gera os embeddings dos enunciados

In [12]:
# Varre todos os enunciado
for idx, id in enumerate(tqdm(id_questoes)):
    texto = enunciado_questoes[idx]

    for nome_modelo, _ in MODELOS_EMB_PARA_GERAR:
        if not existe_embedding(ARQUIVO_EMBEDDINGS_QUESTOES, nome_modelo, idx):
            # TODO: Depois generalizar isso daqui para não ficar tendo que fazer if com o nome dos modelos
            if nome_modelo == NOME_MODELO_EMB_OPENAI:
                emb_gerado = extrair_embeddings_openai(nome_modelo, texto)
                atualizar_embedding(ARQUIVO_EMBEDDINGS_QUESTOES, nome_modelo, idx, emb_gerado)

100%|████████████████████████████████████████████████████████████████████████████| 700/700 [00:00<00:00, 638124.93it/s]


# 3. Pesquisa semântica

## 3.1. Testes com similaridade de cosseno e pandas

In [39]:
def gerar_dict_embeddings(arquivo, coluna_id):
    embeddings = {}

    with h5py.File(arquivo, "r") as f:
        embeddings[coluna_id] = f[coluna_id][:].astype(str)
        for nome_modelo, _ in MODELOS_EMB_NOME_E_DIM_EMB:
            embeddings[nome_modelo] = f[nome_modelo][:]

    return embeddings

In [40]:
dict_emb_chunks = gerar_dict_embeddings(ARQUIVO_EMBEDDINGS_CHUNKS, 'urn')
dict_emb_questoes = gerar_dict_embeddings(ARQUIVO_EMBEDDINGS_QUESTOES, 'id')

df_emb_chunks = pd.DataFrame({
    "urn": dict_emb_chunks["urn"]
})
df_emb_questoes = pd.DataFrame({
    "id": dict_emb_questoes["id"]
})

for nome_modelo, _ in MODELOS_EMB_NOME_E_DIM_EMB:
    df_emb_chunks[nome_modelo] = list(dict_emb_chunks[nome_modelo])
    df_emb_questoes[nome_modelo] = list(dict_emb_questoes[nome_modelo])

In [41]:
def get_chunks_proximos(id_questao, nome_modelo, n_chunks = 20):
    emb_questao = df_emb_questoes[nome_modelo][df_emb_questoes.id == id_questao].values[0]

    df_chunks_similares = df_emb_chunks.copy()
    df_chunks_similares['similarity'] = df_chunks_similares[nome_modelo].apply(lambda x: cosine_similarity([x], [emb_questao])[0,0])
    df_chunks_similares = df_chunks_similares.sort_values("similarity", ascending=False)
    
    return df_chunks_similares.head(n_chunks)

In [44]:
# Para testar:
get_chunks_proximos('1',NOME_MODELO_EMB_OPENAI, 3)

,urn,text-embedding-3-large,similarity
4080,urn:lex:br:federal:lei:2018-08-14;13709!art5_c...,"[-0.04962, 0.02524, -0.000551, 0.002176, 0.031...",0.642434
6964,urn:lex:br:autoridade.nacional.protecao.dados;...,"[-0.04056, 0.02379, 0.001972, -0.01602, 0.0212...",0.637140
4154,urn:lex:br:federal:lei:2018-08-14;13709!art23_...,"[-0.0615, 0.0192, 0.003437, 0.03986, 0.01831, ...",0.633797


## 3.2. Testes com FAISS

Carregar e normalizar embeddings

In [16]:
def carregar_embeddings_faiss(arquivo, coluna_id, nome_modelo):
    with h5py.File(arquivo, "r") as f:
        ids = f[coluna_id][:].astype(str)
        emb = f[nome_modelo][:].astype(np.float32)

    return ids, emb

def normalizar_embeddings(x):
    faiss.normalize_L2(x)
    return x

Criar o índice.

Obs.: acho que os embeddings da OpenAI já são normalizados

In [35]:
mapa_indice_faiss_chunks = {}

for nome_modelo, _ in MODELOS_EMB_NOME_E_DIM_EMB:
    urns, emb_chunks = carregar_embeddings_faiss(ARQUIVO_EMBEDDINGS_CHUNKS, 'urn', nome_modelo)
    emb_chunks = normalizar_embeddings(emb_chunks)
    dim = emb_chunks.shape[1]
    
    index_chunks = faiss.IndexFlatL2(dim)
    index_chunks.add(emb_chunks)

    mapa_indice_faiss_chunks[nome_modelo] = index_chunks
    print(f"Índice criado com {index_chunks.ntotal} vetores")

Índice criado com 7036 vetores


Função de busca no índice

In [34]:
# Mapa id da questão / índice
map_emb_questao_modelo = {}
for nome_modelo, _ in MODELOS_EMB_NOME_E_DIM_EMB:
    ids_q, emb_q = carregar_embeddings_faiss(ARQUIVO_EMBEDDINGS_QUESTOES, 'id', nome_modelo)
    emb_q = normalizar_embeddings(emb_q)
    map_emb_questao_modelo[nome_modelo] = emb_q

In [36]:
# Mapa id da questão / índice
map_id_questao = {id_: i for i, id_ in enumerate(ids_q)}

def get_chunks_proximos_faiss(id_questao, nome_modelo, n_chunks=20):
    idx_q = map_id_questao[id_questao]
    query =  map_emb_questao_modelo[nome_modelo][idx_q:idx_q+1]  # shape (1, dim)
    
    distancias, indices = mapa_indice_faiss_chunks[nome_modelo].search(query, n_chunks)

    resultados = []
    for rank, (i, d) in enumerate(zip(indices[0], distancias[0])):
        resultados.append({
            "rank": rank + 1,
            "urn": urns[i],
            "distancia_l2": float(d),
            "similaridade_cosine_aprox": 1 - d / 2
        })

    return resultados

In [43]:
get_chunks_proximos_faiss('1', NOME_MODELO_EMB_OPENAI, n_chunks=3)

[{'rank': 1,
  'urn': 'urn:lex:br:federal:lei:2018-08-14;13709!art5_cpt_inc6',
  'distancia_l2': 0.7151309251785278,
  'similaridade_cosine_aprox': 0.6424345374107361},
 {'rank': 2,
  'urn': 'urn:lex:br:autoridade.nacional.protecao.dados;conselho.diretor:resolucao:2024-07-16;18;anexo.1!art2_cpt_inc3',
  'distancia_l2': 0.7257207036018372,
  'similaridade_cosine_aprox': 0.6371396481990814},
 {'rank': 3,
  'urn': 'urn:lex:br:federal:lei:2018-08-14;13709!art23_cpt_inc3',
  'distancia_l2': 0.7324053645133972,
  'similaridade_cosine_aprox': 0.6337973177433014}]